In [3]:
import pandas as pd
from kafka import KafkaConsumer
import time
import json
import s3fs

In [4]:
df = pd.read_csv("stock_analysis2.csv")
df.head()

,Index,Date,Open,High,Low,Close,Adj Close,Volume,CloseUSD,unwanted
0,HSI,1986-12-31,2568.300049,2568.300049,2568.300049,2568.300049,2568.300049,0.0,333.879006,6.596165e+06
1,HSI,1987-01-02,2540.100098,2540.100098,2540.100098,2540.100098,2540.100098,0.0,330.213013,6.452109e+06
2,HSI,1987-01-05,2552.399902,2552.399902,2552.399902,2552.399902,2552.399902,0.0,331.811987,6.514745e+06
3,HSI,1987-01-06,2583.899902,2583.899902,2583.899902,2583.899902,2583.899902,0.0,335.906987,6.676539e+06
4,HSI,1987-01-07,2607.100098,2607.100098,2607.100098,2607.100098,2607.100098,0.0,338.923013,6.796971e+06


In [5]:
consumer = KafkaConsumer('test_topic', 
                         bootstrap_servers=['54.235.13.11:9092'], 
                         value_deserializer=lambda x: json.loads(x.decode('utf-8')))

In [6]:
count = 0
for i in consumer:
    print(i)
    if count >=5:
        break
    count += 1

# Output:
# ConsumerRecord(topic='kafkatest', partition=0, offset=109, timestamp=1723015251460, timestamp_type=0, key=None, value={'Index': '000001.SS', 'Date': '2004-04-14', 'Open': 1714.526001, 'High': 1716.819946, 'Low': 1678.33606, 'Close': 1697.156006, 'Adj Close': 1697.156006, 'Volume': 19600.0, 'CloseUSD': 271.54496096}, headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=198, serialized_header_size=-1)

ConsumerRecord(topic='test_topic', partition=0, offset=25, timestamp=1723106679391, timestamp_type=0, key=None, value={'Index': 'N225', 'Date': '1995-08-08', 'Open': 16645.24023, 'High': 16853.5, 'Low': 16512.17969, 'Close': 16838.9707, 'Adj Close': 16838.9707, 'Volume': 0.0, 'CloseUSD': 168.38970700000002, 'unwanted': 280288712.52743125}, headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=222, serialized_header_size=-1)
ConsumerRecord(topic='test_topic', partition=0, offset=26, timestamp=1723106681418, timestamp_type=0, key=None, value={'Index': 'HSI', 'Date': '2008-06-27', 'Open': 21901.26953, 'High': 22201.4707, 'Low': 21773.66992, 'Close': 22042.34961, 'Adj Close': 22042.34961, 'Volume': 2215868800.0, 'CloseUSD': 2865.5054493, 'unwanted': 482755439.8831005}, headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=228, serialized_header_size=-1)
ConsumerRecord(topic='test_topic', partition=0, offset=27, timestamp=1723106683433, timestamp_type=0

In [8]:
s3 = s3fs.S3FileSystem()

In [9]:
for count, i in enumerate(consumer):
    with s3.open(f"s3://kafka-stock-analytics/landing-zone/stock_market_{count}.json", "w") as file:
        json.dump(i.value, file)

KeyboardInterrupt: 